In [1]:
import pandas as pd
import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as log_progress

sns.set(rc={'figure.figsize':(14, 12)})
df = pd.read_csv("dades_per_la_prac2.csv")

In [2]:
df.describe()

,Age (month),days_to_death,project_code,days_to_initial_pathologic_diagnosis,Handedness,days_to_last_followup,days_to_birth,karnofsky_performance_score,year_of_initial_pathologic_diagnosis,init_pathology_dx_method_other,...,T2-Lesion (ml),T2-Lesion percentage,Enhancing (ml),Enhancing percentage,Necrosis (ml),Necrosis percentage,Whole tumor (ml),Whole tumor percentage,Tumor core (ml),Tumor core percentage
count,572.000000,440.000000,0.0,569.0,0.0,556.000000,570.000000,502.000000,570.000000,0.0,...,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000,185.000000
mean,701.480769,474.200000,NaN,0.0,NaN,477.492806,-21322.024561,75.517928,2005.100000,NaN,...,44.191335,3.098778,24.165568,1.658312,3.316259,0.235587,71.673162,4.992677,27.481827,1.893900
std,144.481831,362.080097,NaN,0.0,NaN,390.840201,4393.810158,15.451640,2.965917,NaN,...,34.111949,2.341126,28.488627,1.712522,6.701785,0.491823,52.437659,3.419256,30.643424,1.873750
min,175.000000,5.000000,NaN,0.0,NaN,4.000000,-31419.000000,40.000000,1997.000000,NaN,...,1.057000,0.072771,0.000000,0.000000,0.000000,0.000000,2.568000,0.190906,0.000000,0.000000
25%,654.000000,189.000000,NaN,0.0,NaN,187.000000,-24297.000000,60.000000,2003.000000,NaN,...,16.723000,1.212870,4.058000,0.297696,0.000000,0.000000,31.074000,2.161873,4.758000,0.377472
50%,704.000000,389.500000,NaN,0.0,NaN,350.500000,-21421.000000,80.000000,2005.000000,NaN,...,36.316000,2.443813,16.615000,1.149894,0.466000,0.031789,63.114000,4.298284,20.066000,1.465181
75%,800.250000,631.000000,NaN,0.0,NaN,668.000000,-19884.000000,80.000000,2007.000000,NaN,...,65.723000,4.780894,34.430000,2.503915,2.968000,0.195677,104.604000,7.569745,37.769000,2.847438
max,1033.000000,1987.000000,NaN,0.0,NaN,1757.000000,-5309.000000,100.000000,2011.000000,NaN,...,186.548000,11.597724,239.957000,9.387060,41.852000,3.397925,307.187000,14.909095,247.126000,9.667509


In [3]:
style = {'description_width': 'initial'}
possible_meas = ["T2-Lesion (ml)",
                 "T2-Lesion percentage",
                 "Enhancing (ml)",
                 "Enhancing percentage",
                 "Necrosis (ml)",
                 "Necrosis percentage",
                 "Whole tumor (ml)",
                 "Whole tumor percentage",
                 "Tumor core (ml)",
                 "Tumor core percentage"]
translate_x = {
    "Age in years": "Age (year)", "Days to death": "days_to_death", 
               "Days to last follow-up": "days_to_last_followup", 
               "Age at Initial Pathologic Diagnosis": "age_at_initial_pathologic_diagnosis"}

df["Age (year)"] = df["Age (month)"] / 12
def draw_plot(selectX, selectY, _all, disable_histogram,):
    if _all:
        fig, ax = plt.subplots(len(possible_meas))
        fig.set_size_inches(10, 80)

        build_loop = list()
        for meas in possible_meas:   
            build_loop.append([selectX, meas])
        print("Loading plots...")
        k = 0
        for var_x, meas in log_progress(build_loop):
            index1 = 0
            index2 = possible_meas.index(meas)
            if not disable_histogram:
                sns.histplot(
                    data=df, 
                    x=translate_x[var_x], 
                    y=meas, 
                    hue="Sex", 
                    ax=ax[index2],                    
                )
            sns.regplot(
                data=df[df["Sex"] == "male"], 
                x=translate_x[var_x], 
                y=meas,
                color=sns.color_palette()[0], 
                ax=ax[index2],# scatter_kws={'s':2}
            )
            sns.regplot(
                data=df[df["Sex"] == "female"], 
                 x=translate_x[var_x], 
                y=meas,
                color=sns.color_palette()[1], 
                ax=ax[index2],# scatter_kws={'s':2}
            )
            k += 1
        plt.subplots_adjust(bottom=-0.2, top=1.3)

    else:
        if not disable_histogram:
            sns.histplot(
                data=df, 
                x=translate_x[selectX], 
                y=selectY, 
                hue="Sex" 
            )
        sns.regplot(
            data=df[df["Sex"] == "male"], 
            x=translate_x[selectX],  
            y=selectY,
            color=sns.color_palette()[0], label='male'
        )
        sns.regplot(
            data=df[df["Sex"] == "female"], 
            x=translate_x[selectX],  
            y=selectY, label='female',
            color=sns.color_palette()[1]
        )
        plt.legend()
    
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)


In [4]:
widgets.interact(
    draw_plot,
    _all=widgets.Checkbox(value=False, description='Select all Y variables', disabled=False),    
    selectX=widgets.Dropdown(options=list(translate_x.keys()), description="Select X"),
    selectY=widgets.Dropdown(options=possible_meas, description="Select Y"),
    disable_histogram=widgets.Checkbox(value=False, description='Disable square histogram', disabled=False)
)
print()

interactive(children=(Dropdown(description='Select X', options=('Age in years', 'Days to death', 'Days to last…